<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/explore_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/SourCherries/VAE-GAN-CelebA.git

fatal: destination path 'VAE-GAN-CelebA' already exists and is not an empty directory.


In [13]:
import sys, getopt
if sys.version_info < (3,4):
    print('You are running an older version of Python!\n\n',
          'You should consider updating to Python 3.4.0 or',
          'higher.\n')

In [ ]:
sys.path

In [19]:
sys.path.insert(0,'/VAE-GAN-CelebA/libs')
sys.path.insert(0,'/libs')

['/libs',
 '/VAE-GAN-CelebA/libs',
 '/libs',
 '/VAE-GAN-CelebA/libs',
 '/libs',
 '/VAE-GAN-CelebA/libs',
 '/VAE-GAN-CelebA/libs',
 '/VAE-GAN-CelebA/libs',
 '/content/VAE-GAN-CelebA/libs',
 '/content/VAE-GAN-CelebA/libs',
 '/content/VAE-GAN-CelebA/libs',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [17]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import libs.make_network as make_network
import libs.utils as utils
from scipy.io import loadmat, savemat

print('TF version = ',tf.__version__)

ModuleNotFoundError: ignored

In [ ]:
file = "example.jpg"
print ('Input file is ', file)

In [ ]:
sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()
if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()

In [ ]:
img = plt.imread(file)[..., :3]
img = utils.preprocess128(img,crop_factor=0.8)[np.newaxis]

In [ ]:
#generate images from z
z = sess.run(Z_mu, feed_dict={X: img, is_training: False})

In [ ]:
#save data in Matlab format
savemat(file[:-4]+'_z',dict(latent=z))